In [1]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode
import Data.Attoparsec.ByteString.Char8 as A

input <- dayLines 18

In [2]:
type Expr = [Value]
data Value = Number Int | Op Char | Paren Expr deriving (Eq, Show)

In [3]:
import Control.Applicative ((<|>))
import qualified Data.ByteString as BS

parseExpr :: A.Parser Expr
parseExpr = A.many' (parseValue <* A.skipSpace)
    where
        parseValue = A.choice
            [ Number <$> A.decimal
            , Op <$> (A.char '+' <|> A.char '*')
            , "(" *>  (Paren <$> parseExpr) <* ")"]

In [4]:
expressions = map (parsed parseExpr) input

In [5]:
evalValue1 :: Value -> Int
evalValue1 (Number n) = n
evalValue1 (Paren p) = evalExpr1 p

evalExpr1 :: Expr -> Int
evalExpr1 [value] = evalValue1 value
evalExpr1 (l:o:r:rest) = let
    lE = evalValue1 l
    rE = evalValue1 r
    in case o of
        Op '+' -> evalExpr1 (Number (lE+rE):rest)
        Op '*' -> evalExpr1 (Number (lE*rE):rest)

In [6]:
sum $ map evalExpr1 expressions

4696493914530

In [7]:
evalValue2 :: Value -> Int
evalValue2 (Number n) = n
evalValue2 (Paren p) = evalExpr2 p

performAdditions :: Expr -> Expr
performAdditions [value] = [value]
performAdditions (l:o:r:rest) = let
    lE = evalValue2 l
    rE = evalValue2 r
    in case o of
        Op '+' -> performAdditions (Number (lE+rE):rest)
        Op '*' -> l:o:performAdditions (r:rest)

performMultiplications :: Expr -> Int
performMultiplications [value] = evalValue2 value
performMultiplications (l:o:r:rest) = let
    lE = evalValue2 l
    rE = evalValue2 r
    in case o of
        Op '*' -> performMultiplications (Number (lE*rE):rest)

evalExpr2 :: Expr -> Int
evalExpr2 = performMultiplications . performAdditions

In [8]:
sum $ map evalExpr2 expressions

362880372308125